# 🚀 Part 8: Deploying AI Agents to Vertex AI Agent Engine
## Content Creation Studio Workshop

Welcome to Part 8 of **Google ADK: From Local Development to Vertex AI Deployment**! You've built a sophisticated multi-agent system locally. Now comes the pivotal transition: **deploying to the cloud for learning and demonstration**.

**⚠️ Important: Learning vs. Production**

This tutorial teaches cloud deployment for **learning, demonstration, and testing purposes**. The approach shown here gets your agent system running in the cloud quickly, but is **not production-ready**. Real production systems require additional features not covered here:

- **Comprehensive testing** (unit tests, integration tests, end-to-end tests)
- **Model evaluation** and validation pipelines
- **CI/CD automation** for safe deployments
- **Advanced monitoring** and alerting systems
- **Load testing** and performance optimization
- **Security hardening** and compliance measures
- **Disaster recovery** and backup strategies

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- Part 2: Giving Your Agent Custom Tools ✅
- Part 3: Building Agent Teams with Agent-as-a-Tool ✅
- Part 4: Multi-Step Workflows with SequentialAgent ✅
- Part 5: Building Iterative Workflows with LoopAgent ✅
- Part 6: Efficient Workflows with ParallelAgent ✅
- Part 7: The Capstone Project ✅
- **Part 8: Deploying to Vertex AI Agent Engine** (You are here) 🎯
- Part 9: Full-Stack Deployment with Cloud Run

---

## 🎯 What You'll Learn

In this notebook, you'll learn:
- **Why deployment matters** - Moving from prototype to cloud deployment
- **Vertex AI Agent Engine** - Managed, scalable AI agent infrastructure
- **GCP Setup** - Using `setup_gcp.sh` to configure your environment
- **Deploying your agents** - Your 11-agent system in the cloud for demo/learning
- **Understanding the endpoint** - What you get and how it works
- **Bridge to Part 9** - How this connects to the full-stack app

---

## ⚠️ Prerequisites

Before starting, ensure you have:
1. ✅ Google Cloud Project with **billing enabled**
2. ✅ Completed Parts 1-7 of this tutorial
3. ✅ Your Content Creation Studio code
4. ✅ Google Cloud SDK (`gcloud`) installed (for local execution)

**Don't have these?** We'll guide you through the setup!

---

## 🤔 Why Deployment Matters

### The Local Development Trap

Your Content Creation Studio works perfectly locally. You run it in Jupyter notebooks:

```python
# Local execution
async def create_content():
    session = await session_service.create_session(...)
    await run_agent_query(coordinator_agent, query, session, user_id)
```

**But this has fundamental limitations:**

❌ **Single User** - Only you can use it  
❌ **No Persistence** - Restart = lose everything (InMemorySessionService)  
❌ **Your Computer** - Requires your machine to be running  
❌ **No Scalability** - Can't handle multiple requests  
❌ **No Monitoring** - Can't track performance or errors  
❌ **No API** - Can't integrate with web apps or other services  
❌ **No Reliability** - Crashes stop everything  

### Cloud Deployment Requirements

To serve users through the cloud, you need:

✅ **Accessible 24/7** - Available from anywhere, anytime  
✅ **Scalable** - Handles 1 or 1,000 concurrent users  
✅ **Reliable** - Automatic restarts, error recovery  
✅ **Secure** - Authentication, authorization, data encryption  
✅ **Fast** - Low latency, optimized performance  
✅ **Observable** - Logs, metrics, tracing  
✅ **API-First** - HTTP endpoints for integration  
✅ **Cost-Effective** - Pay for what you use  

**This is what cloud deployment enables.**

**⚠️ Note:** This tutorial shows basic cloud deployment for learning and demonstration. Production systems require comprehensive testing, CI/CD, evaluation frameworks, and advanced monitoring not covered here.

---

## 🌟 Introducing Vertex AI Agent Engine

### What is Agent Engine?

**Vertex AI Agent Engine** is Google Cloud's fully-managed platform for deploying and running Google ADK agents at scale. Think of it as "Cloud Run for AI Agents."

> "Vertex AI Agent Engine is a fully managed service that allows you to deploy reasoning engines (ADK agents) to a scalable, serverless infrastructure. It handles infrastructure, auto-scaling, monitoring, and provides a gRPC API for integration."
> 
> — Google Cloud Documentation

**In simple terms:** You give Google your agent code, and they handle everything else.

---

### The 6 Powers of Agent Engine

#### 1. Zero Infrastructure Management

You don't set up:
- Virtual machines or Kubernetes clusters
- Container orchestration
- Load balancers
- Auto-scaling rules
- Health checks
- Network configuration

**Google handles all of it.** You focus on your agent logic.

#### 2. Serverless Auto-Scaling

```
1 user  → 1 instance  → $X
10 users → 2 instances → $2X  (automatically scales up)
1 user  → 1 instance  → $X   (automatically scales down)
0 users → 0 instances → $0   (scales to zero!)
```

**Pay only for actual usage.** No idle servers burning money.

#### 3. Managed Reliability Features

✅ **Automatic restarts** - Agent crashes? Restarted instantly  
✅ **Health monitoring** - Continuous health checks  
✅ **Multi-zone deployment** - High availability across data centers  
✅ **Versioning** - Deploy new versions without downtime  
✅ **Rollback** - Instant rollback to previous versions  

#### 4. Built-in Observability

✅ **Cloud Logging** - All agent logs automatically collected  
✅ **Cloud Monitoring** - Performance metrics, latency, errors  
✅ **Cloud Trace** - Request tracing through your agent system  
✅ **Debuggable** - Full visibility into deployed behavior  

#### 5. Secure by Default

✅ **IAM integration** - Fine-grained access control  
✅ **Service accounts** - Secure authentication  
✅ **VPC integration** - Private network deployment  
✅ **Encryption** - Data encrypted in transit and at rest  

#### 6. Seamless ADK Integration

Here's the magic: **Your local ADK agent code deploys with minimal changes.**

```python
# Local execution
agent = create_content_creation_coordinator()
runner = Runner(agent=agent, session_service=session_service)
await runner.run_async(...)

# Cloud deployment - SAME agent code!
deployed_agent = agent.deploy(
    project="my-project",
    location="us-central1"
)
# Done! Now accessible via gRPC API
```

The agent you built locally **just works** in the cloud.

---

### Agent Engine vs Building Your Own

| Concern | DIY Deployment | Agent Engine |
|---------|----------------|-------------|
| Infrastructure | Set up VMs, containers, orchestration | ✅ Managed |
| Scaling | Configure auto-scaling policies | ✅ Automatic |
| Monitoring | Set up logging, metrics, dashboards | ✅ Built-in |
| Security | Configure firewalls, IAM, encryption | ✅ Default |
| Updates | Blue-green deployments, rollbacks | ✅ One command |
| Cost (idle) | Pay for idle servers | ✅ $0 at zero usage |
| Time to Deploy | Days to weeks | ✅ Minutes |
| Maintenance | Ongoing ops work | ✅ Google manages |

**Agent Engine eliminates months of DevOps work** for basic cloud deployment.

**⚠️ Note:** While Agent Engine simplifies infrastructure, production systems still need comprehensive testing, CI/CD, evaluation, and advanced monitoring that you must implement separately.

---

## 🛠️ Setting Up Google Cloud Environment

Before deploying, we need to prepare your Google Cloud environment. We provide a comprehensive setup script that handles everything automatically.

### What Does `setup_gcp.sh` Do?

**📂 Location:** `content_creation_mas/deployment/setup_gcp.sh`

This script automates your entire GCP setup:

**1. Environment Configuration**
- Loads your `.env` file
- Validates `GOOGLE_CLOUD_PROJECT` and `GOOGLE_CLOUD_LOCATION`
- Sets gcloud defaults

**2. Enables Required APIs**
```bash
✓ aiplatform.googleapis.com           # Vertex AI / Agent Engine
✓ run.googleapis.com                  # Cloud Run (Part 9)
✓ cloudbuild.googleapis.com           # Cloud Build
✓ artifactregistry.googleapis.com     # Docker registry
✓ storage.googleapis.com              # Cloud Storage
✓ iam.googleapis.com                  # IAM
✓ cloudresourcemanager.googleapis.com # Resource Manager
```

**3. Creates Artifact Registry Repository**
- Repository name: `content-studio`
- Location: `{region}-docker.pkg.dev/{project}/content-studio`
- Stores your Cloud Run container images (Part 9)

**4. Configures Docker Authentication**
- Authenticates Docker with Artifact Registry
- Allows pushing images

**5. Creates Service Account**
- Name: `content-studio-sa`
- Email: `content-studio-sa@{project}.iam.gserviceaccount.com`
- Runs your deployed services

**6. Grants IAM Roles**
- `roles/aiplatform.user` - Access Agent Engine
- `roles/run.invoker` - Invoke Cloud Run services
- `roles/storage.objectViewer` - Read from Cloud Storage
- `roles/logging.logWriter` - Write logs

**7. Creates Cloud Storage Bucket**
- Bucket: `gs://{project}-content-studio`
- Used for Agent Engine staging

### Running the Setup Script

**If you haven't already**, run this once from your terminal:

```bash
# Navigate to deployment directory
cd content_creation_mas/deployment

# Create .env file
cat > .env << EOF
GOOGLE_CLOUD_PROJECT=your-project-id
GOOGLE_CLOUD_LOCATION=us-central1
GOOGLE_API_KEY=your-api-key
EOF

# Make executable and run
chmod +x setup_gcp.sh
./setup_gcp.sh
```

**The script is interactive** and will show you what it will do before proceeding.

---

## 📦 Step 1: Install Required Packages

In [ ]:
!pip install google-adk==1.19.0 -q
!pip install -q google-cloud-aiplatform python-dotenv

print("✅ Packages installed successfully!")

## 🔑 Step 2: Set Up Environment Variables

Load your Google Cloud configuration from the `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

# Get GCP configuration
PROJECT_ID = os.environ.get('GOOGLE_CLOUD_PROJECT')
LOCATION = os.environ.get('GOOGLE_CLOUD_LOCATION', 'us-central1')
STAGING_BUCKET = os.environ.get('GOOGLE_CLOUD_STORAGE_BUCKET')

print("📋 Your Configuration:")
print(f"   Project ID: {PROJECT_ID}")
print(f"   Location: {LOCATION}")
print(f"   Staging Bucket: {STAGING_BUCKET}")

if not PROJECT_ID:
    raise ValueError("❌ GOOGLE_CLOUD_PROJECT environment variable is required")

print("\n✅ Environment configured!")

## 🔐 Step 3: Authenticate with Google Cloud

This authenticates your notebook with Google Cloud so you can deploy agents.

In [ ]:
# Authenticate with Google Cloud
# In Colab, use google.colab.auth
# In local environment, use gcloud auth application-default login

try:
    from google.colab import auth
    auth.authenticate_user()
    print("✅ Authenticated with Google Cloud (Colab)")
except ImportError:
    # Running locally
    print("🏠 Running locally.")
    print("   Make sure you've run: gcloud auth application-default login")
    print("   Or set GOOGLE_APPLICATION_CREDENTIALS environment variable")

## ⚙️ Step 4: Initialize Vertex AI

Initialize Vertex AI with your project configuration.

In [ ]:
import vertexai

# Initialize Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET
)

print("✅ Vertex AI initialized")
print(f"   Project: {PROJECT_ID}")
print(f"   Region: {LOCATION}")
print(f"   Staging Bucket: {STAGING_BUCKET}")

## ✅ Step 5: Verify Required APIs Are Enabled

Check that the necessary Google Cloud APIs are enabled. If `setup_gcp.sh` ran successfully, these should all be enabled.

In [ ]:
# Check if required APIs are enabled
print("Checking enabled APIs...\n")
!gcloud services list --enabled --filter="name:(aiplatform.googleapis.com OR storage.googleapis.com OR run.googleapis.com)" --project={PROJECT_ID}

print("\n✅ If you see the APIs listed above, you're ready to deploy!")
print("❌ If APIs are missing, run: ./deployment/setup_gcp.sh")

## 🤖 Step 6: Prepare Your Agent for Deployment

Load your Content Creation Coordinator agent that we built in Part 7.

In [ ]:
import sys
sys.path.append('../backend')

from agents.content_creation_coordinator import create_content_creation_coordinator

# Create the agent
agent = create_content_creation_coordinator()

print("✅ Agent created successfully!")
print(f"   Agent name: {agent.name}")
print(f"   Description: {agent.description}")
print(f"\n   This is your 11-agent Content Creation Studio!")

## 🚀 Step 7: Deploy Agent to Vertex AI Agent Engine

This is the big moment! This step will:
1. Package your agent code and dependencies
2. Upload to Agent Engine
3. Configure auto-scaling and monitoring
4. Make it accessible via gRPC API

**⏱️ This takes 5-10 minutes** - Google is setting up your production infrastructure!

In [ ]:
print("🚀 Deploying agent to Vertex AI Agent Engine...")
print("   This may take 5-10 minutes...")
print("   ☕ Perfect time for a coffee break!\n")

# Deploy the agent
deployed_agent = agent.deploy(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

print("\n" + "="*60)
print("🎉 AGENT DEPLOYED SUCCESSFULLY!")
print("="*60)
print(f"\n📍 Agent Resource Name:")
print(f"   {deployed_agent.resource_name}")
print(f"\n💡 IMPORTANT: Save this resource name!")
print(f"   This is your agent's cloud deployment endpoint.")
print(f"   Your backend (Part 9) will use this to connect.")
print(f"\n⚠️  Note: This deployment is for learning/demo purposes.")
print(f"   Production deployments require additional features like testing, CI/CD, etc.")

# Save to environment variable file
with open('../.env', 'a') as f:
    f.write(f"\nAGENT_ENGINE_RESOURCE_NAME={deployed_agent.resource_name}\n")

print(f"\n✅ Resource name saved to .env file")
print(f"\n🔗 Your agent is now live and accessible via API!")

## 🔍 Understanding What You Just Deployed

### The Agent Engine Endpoint

The **resource name** you just received is your agent's production endpoint:

```
projects/123456789/locations/us-central1/reasoningEngines/987654321
           ↑                 ↑                      ↑
      Project ID          Region            Agent Instance ID
```

This endpoint:
- ✅ Is globally unique
- ✅ Is accessible via gRPC API
- ✅ Handles authentication via IAM
- ✅ Auto-scales based on demand
- ✅ Is monitored and logged automatically

### How the Endpoint Works

```
┌─────────────────────┐
│  Your Application   │  (Backend server, Cloud Run, etc.)
│  (Part 9)           │
└──────────┬──────────┘
           │
           │ gRPC API call with:
           │ - resource_name
           │ - user_query
           │ - session_id
           ↓
┌─────────────────────────────────────────┐
│  Vertex AI Agent Engine                 │
│  projects/.../reasoningEngines/...      │
│                                         │
│  ┌────────────────────────────────┐    │
│  │  Your Content Creation Studio  │    │
│  │  - 11 Specialist Agents        │    │
│  │  - Sequential/Parallel/Loop    │    │
│  │  - Session Management          │    │
│  └────────────────────────────────┘    │
│                                         │
│  Auto-scaling: 0-N instances           │
│  Monitoring: Logs, Metrics, Traces     │
└─────────────────┬───────────────────────┘
                  │
                  │ Calls Gemini API
                  ↓
         ┌──────────────────┐
         │   Gemini 2.5     │
         └──────────────────┘
```

**Everything in between is managed by Google!**

---

## 🧪 Step 8: Test Your Deployed Agent

Now let's verify your deployed agent works by creating content!

In [ ]:
# Connect to your deployed agent
# (If you just deployed, use deployed_agent from above)

remote_agent = deployed_agent

print(f"✅ Connected to deployed agent")
print(f"   Resource: {remote_agent.resource_name}")

## 📝 Step 9: Create a Session and Test Content Creation

Let's test the deployed agent by creating content about sustainable living!

In [ ]:
import asyncio
from IPython.display import display, Markdown

# Create a session
user_id = "test_user_001"
session = await remote_agent.async_create_session(user_id=user_id)
session_id = session['id']

print(f"✅ Session created: {session_id}\n")

# Test query
query = """Create a complete content package for:
- Topic: Sustainable Fashion for Modern Consumers
- Target Audience: Environmentally conscious millennials
- Tone: Inspirational and Educational
- Keywords: eco-friendly, sustainable, ethical fashion, slow fashion
"""

print("🚀 Sending query to your deployed agent...\n")
print("📋 Query:")
print(query)
print("\n" + "="*60)
print("📝 AGENT RESPONSE:")
print("="*60 + "\n")

# Stream the response
full_response = ""
async for event in remote_agent.async_stream_query(
    user_id=user_id,
    session_id=session_id,
    message=query
):
    if isinstance(event, dict):
        content = event.get("content", event.get("parts", {}))
        
        if isinstance(content, dict):
            parts = content.get("parts", [])
            for part in parts:
                if isinstance(part, dict) and part.get("text"):
                    text = part["text"]
                    full_response += text
                    print(text, end="", flush=True)
        
        if "text" in event:
            text = event["text"]
            full_response += text
            print(text, end="", flush=True)

print("\n\n" + "="*60)
print("✅ CONTENT CREATION COMPLETED!")
print("="*60)
print(f"\n🎉 Your deployed agent just created a complete content package!")
print(f"   - 11 agents worked together")
print(f"   - In the cloud, not on your laptop")
print(f"   - Auto-scaled based on demand")
print(f"   - Basic monitoring and logging enabled")
print(f"\n⚠️  Note: This is a demo deployment. Production would need comprehensive")
print(f"   testing, CI/CD, evaluation frameworks, and advanced monitoring.")

## 📊 Step 10: Monitor Your Deployed Agent

View logs and performance metrics from your production agent.

In [ ]:
# View recent logs from your agent
print("📋 Recent agent logs from production:\n")

!gcloud logging read "resource.type=aiplatform.googleapis.com/ReasoningEngine" \
    --limit=10 \
    --project={PROJECT_ID} \
    --format="table(timestamp, severity, textPayload)"

print("\n✅ These are real cloud logs from your deployed agent!")
print("⚠️  Note: Production systems need advanced monitoring, alerting, and log analysis.")

## 📋 Step 11: List All Your Deployed Agents

See all agents you have deployed to Agent Engine.

In [ ]:
print("🤖 Your deployed agents:\n")

!gcloud beta ai reasoning-engines list \
    --location={LOCATION} \
    --project={PROJECT_ID}

print("\n💡 Each agent has a unique resource name that serves as its endpoint.")

## 🌉 The Bridge to Part 9: Full-Stack Deployment

### What We Have Now

After completing Part 8, you have:

✅ **Agent Engine Endpoint** - Your multi-agent system running in the cloud  
✅ **Resource Name** - The API identifier to access it  
✅ **Auto-scaling Infrastructure** - Handles varying load  
✅ **Basic Monitoring** - Logs and metrics  
✅ **gRPC API** - Programmatic access  

### What's Missing

But users can't interact with it yet because:

❌ No user interface (UI)  
❌ No web backend to handle HTTP requests  
❌ No authentication flow  
❌ No session management for web users  
❌ No public URL  

**This is what Part 9 solves.**

### Part 9 Preview: The Complete Stack

In Part 9, we'll deploy:

**Frontend (React App)**
- User-friendly web interface
- Content creation forms
- Real-time progress updates
- Results display

**Backend (FastAPI Server)**
- HTTP REST API endpoints
- Connects to Agent Engine using the resource name
- Handles web requests and streams responses
- Session management for web users

**Architecture:**
```
User's Browser
    ↓
Cloud Run (Frontend + Backend)
    ↓ (using AGENT_ENGINE_RESOURCE_NAME)
Agent Engine (Your Agents) ← You are here!
    ↓
Gemini API
```

**The critical connection:** Your backend uses the `AGENT_ENGINE_RESOURCE_NAME` from Part 8!

```python
# Backend code (Part 9)
import os
from google import genai

# This resource name from Part 8!
AGENT_ENGINE_RESOURCE_NAME = os.getenv("AGENT_ENGINE_RESOURCE_NAME")

client = genai.Client(
    vertexai=True,
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION")
)

# Connect to your Agent Engine endpoint
agent = client.agentic.get_agent(AGENT_ENGINE_RESOURCE_NAME)

# Now users can access your agents via web UI!
response = agent.query(user_query=request.topic)
```

**Part 8 provides the AI backend.**  
**Part 9 provides the user-facing application.**

Together, they form a complete cloud-deployed system accessible to anyone with a web browser.

**⚠️ Important:** Both Part 8 and Part 9 demonstrate cloud deployment for learning purposes. For production, you'd need to add comprehensive testing, CI/CD, evaluation, advanced monitoring, security hardening, and disaster recovery.

---

## 🔄 Step 12: Updating Your Agent (Optional)

When you make changes to your agent, you can redeploy to update the production version.

In [ ]:
# After making changes to your agent code:
# 1. Recreate the agent with your changes
updated_agent = create_content_creation_coordinator()

# 2. Deploy the updated version
print("🔄 Updating deployed agent...")
print("   This may take 5-10 minutes...\n")

updated_deployment = updated_agent.deploy(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

print("\n✅ Agent updated successfully!")
print(f"   Resource name: {updated_deployment.resource_name}")
print("\n💡 The resource name remains the same for updates.")

## 🗑️ Step 13: Delete Agent (Cleanup - Optional)

When you're done testing or want to remove the agent, you can delete it.

**⚠️ WARNING: This permanently deletes your deployed agent!**

In [ ]:
# Uncomment to delete:

# Extract agent ID from resource name
# AGENT_ID = deployed_agent.resource_name.split('/')[-1]

# !gcloud beta ai reasoning-engines delete {AGENT_ID} \
#     --location={LOCATION} \
#     --project={PROJECT_ID} \
#     --quiet

print("⚠️  To delete an agent, uncomment the code above.")
print("💡 Tip: Keep your agent deployed if you're proceeding to Part 9!")

## 🎉 Congratulations!

### What You Accomplished

✅ **Understood why deployment matters** - Moving from prototype to cloud deployment  
✅ **Learned about Agent Engine** - Managed, scalable AI agent infrastructure  
✅ **Set up Google Cloud** - Using `setup_gcp.sh` to configure everything  
✅ **Deployed your agents** - 11-agent system running in the cloud for demo/learning  
✅ **Got your endpoint** - Agent Engine resource name for integration  
✅ **Tested deployment** - Verified your agent works in the cloud  
✅ **Monitored logs** - Viewed deployed agent behavior  

**⚠️ Important:** This deployment is for learning and demonstration. Production systems require comprehensive testing, CI/CD, evaluation frameworks, advanced monitoring, security hardening, and disaster recovery.

### Core Concepts Mastered

1. **🆕 Cloud Deployment** - Transforming local code to cloud-hosted service
2. **🆕 Vertex AI Agent Engine** - Managed platform for ADK agents
3. **🆕 Agent Engine Endpoint** - Resource name as the API identifier
4. **🆕 GCP Setup** - Automated environment configuration
5. **🆕 Monitoring & Observability** - Cloud logging and metrics for deployed agents

### The Deployment Stack

```
Part 8: Agent Engine ← You are here! ✅
    ↓ (provides AI backend endpoint)
Part 9: Cloud Run Frontend + Backend ← Next!
    ↓ (provides web interface)
Users
```

---

## 🚀 What's Next?

Your AI agents are now running in the cloud! But users still can't interact with them through a web interface.

**In Part 9: Full-Stack Deployment with Cloud Run**, we'll complete the stack:

- 🎨 **React Frontend** - Beautiful web UI for content creation
- ⚙️ **FastAPI Backend** - REST API that connects to Agent Engine
- 🐳 **Docker Containerization** - Package frontend + backend
- ☁️ **Cloud Run Deployment** - Serverless hosting with auto-scaling
- 🔗 **Complete Integration** - Users → Web App → Agent Engine → Gemini

### Preview of Part 9

```bash
cd deployment
./deploy-cloudrun.sh

# Output:
# ✓ Frontend built
# ✓ Backend packaged
# ✓ Docker image created
# ✓ Pushed to Artifact Registry
# ✓ Deployed to Cloud Run
#
# 🎉 Your app is live!
# URL: https://content-studio-xyz.run.app
```

**The final piece of the puzzle!**

**⚠️ Remember:** Parts 8-9 show cloud deployment for learning/demo. Before production, add comprehensive testing, CI/CD, evaluation, monitoring, security, and disaster recovery.

---

## 📚 Resources

- [Vertex AI Agent Engine Documentation](https://cloud.google.com/vertex-ai/docs/agent-engine)
- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [Cloud Run Documentation](https://cloud.google.com/run/docs)
- [GitHub Repository](https://github.com/Saoussen-CH/content_creation_mas_workshop)

---

**🎓 End of Part 8**

**Next:** [Part 9 - Full-Stack Deployment with Cloud Run](https://colab.research.google.com/github/Saoussen-CH/content_creation_mas_workshop/blob/main/content_creation_mas/notebooks/part9_fullstack_deployment.ipynb)